In [1]:
using Distributions
using Random
using Statistics
using StatsFuns
using LinearAlgebra
using Plots
using PyCall

In [2]:
function Zt(p, zt, n)
    return [i > 2 * p ? 0 : (i < p ? zt : -zt) for i in (rand() for i in 1:n)]
end

Zt (generic function with 1 method)

In [3]:
function MA1_process(n, theta, sigma)
    Z = rand(Normal(0, sigma), n+1)
    return [Z[i+1] + theta*Z[i] for i in 1:n]
end

MA1_process (generic function with 1 method)

In [4]:
function theoretic_ACF_MA1(theta, lag)
    acf = zeros(1, lag[2] + 1)
    acf[1] = 1
    acf[2] = theta/(1+theta^2)
    return acf
end

theoretic_ACF_MA1 (generic function with 1 method)

In [5]:
function autocov(X, lag)
    return [1/length(X) * (sum((X[1:end - i] .- mean(X)) .* (X[i + 1:end] .- mean(X)))) for i in lag[1]:lag[2]]
end

autocov (generic function with 1 method)

In [6]:
function autocorr(X, lag)
    return [i/autocov(X, [0, 1])[1] for i in autocov(X, lag)]
end

autocorr (generic function with 1 method)

In [7]:
@time begin
for i in 1:10000
    Xt = MA1_process(1000, 0.5, 1)
    autocorr(Xt, [0, 2])
end
end

  1.760131 seconds (380.00 k allocations: 2.203 GiB, 19.83% gc time)


In [8]:
function Q(X, k)
    values = [abs(X[j] - X[i]) for i in 1:length(X) for j in 1:i]
    values = sort(values)
    return k * values[Int64(floor((binomial(length(X), 2) + 2) / 4))]
end
Xt = MA1_process(100, 0.5, 1)
println(Q(Xt, 2.2191))

0.9389803576695295


In [9]:
function autocorr1(X, lag, k)
    acf = Float64[]
    for h in lag[1]:lag[2]
        u = X[1:end-h]
        v = X[h+1:end]
        append!(acf, ((Q(u .+ v, k)) ^ 2 - (Q(u .- v, k)) ^ 2) / ((Q(u .+ v, k)) ^ 2 + (Q(u .- v, k)) ^ 2))
    end
    return acf
end

autocorr1 (generic function with 1 method)

In [10]:
Xt = MA1_process(1000, 0.5, 1)
println(autocorr1(Xt, [0, 5], 2.2191))

[1.0, 0.47929399941799916, 0.09267493976310583, -0.007836117022249836, -0.009704022948241324, 0.010234336584157021]


#########################

In [11]:
function g(alfa, n)
    return Int64(floor(n * alfa))
end

g (generic function with 1 method)

In [12]:
function L(X, t, alfa)
    Y = sort(X)
    n = length(X)
    g_ = g(alfa, n)
    return (Y[g_] < X[t] && X[t] < Y[n - g_]) ? 1 : 0
end

L (generic function with 1 method)

In [13]:
function mean_X(X, alfa)
    suma1 = sum([L(X, i, alfa) for i in 1:length(X)])
    suma2 = sum([X[i] .* L(X, i, alfa) for i in 1:length(X)])
    return 1 / suma1 * suma2
end

mean_X (generic function with 1 method)

In [14]:
function autocov2(X, lag, alfa)
    n = length(X)
    L_vector = [L(X, i, alfa) for i in 1:n]
    acvf = Float64[]
    for h in lag[1]:lag[2]
        if h == 0
            suma1 = sum(L_vector)
            suma2 = sum(X .^ 2 .* L_vector)
        else
            suma1 = sum(L_vector[1:end-h] .* L_vector[h + 1:end])
            suma2 = sum((X[1:end-h] .- mean_X(X, alfa)) .* (X[1+h:end] .- mean_X(X, alfa)) .* L_vector[1:end-h] .* L_vector[1+h:end])
        end
        append!(acvf, suma2 / suma1)
        end
    return acvf
end

autocov2 (generic function with 1 method)

In [15]:
function autocorr2(X, lag, alfa)
    acvf = autocov2(X, lag, alfa)[1]
    return [i / acvf for i in autocov2(X, lag, alfa)]
end

autocorr2 (generic function with 1 method)

In [16]:
Xt = MA1_process(1000, 0.5, 1)
println(autocorr2(Xt, [0, 5], 0.015))

[1.0, 0.35027820173515645, 0.015078664301351486, -0.006251664326427437, -0.03844526416151921, -0.04453018059475347]


#############################

In [17]:
function autocorr4(X, lag)
    X = X .- mean(X)
    acf = Float64[1.0]
    denominator = median([i^2 for i in X])
    for h in lag[1] + 1:lag[2]
        append!(acf, median(X[1:end-h] .* X[1+h:end])/denominator)
    end
    return acf
end

autocorr4 (generic function with 1 method)

In [18]:
Xt = MA1_process(1000, 0.5, 1)
println(autocorr4(Xt, [0, 5]))

[1.0, 0.2988127695321756, 0.025330414476251322, -0.01276365200530693, -0.006387238734640111, -0.002770354968405067]


######################################

In [19]:
function J(x)
    return norminvcdf(x)
end

J (generic function with 1 method)

In [20]:
function c(X)
    n = length(X)
    R = sort(X)
    return 1 / sum([(J((findfirst(item -> item == R[i], X)) / (n + 1)))^2 for i in 1:n])
end

c (generic function with 1 method)

In [21]:
function autocorr3(X, lag)
    acf = Float64[]
    n = length(X)
    R = sort(X)
    for h in lag[1]:lag[2]
        suma = 0
        for i in 1:n-h
            suma += J((findfirst(item -> item == X[i], R))/ (n+1)) * J((findfirst(item -> item == X[i+h], R))/ (n+1))
        end
        append!(acf, c(X) * suma)
    end
    return acf
end

autocorr3 (generic function with 1 method)

In [22]:
Xt = MA1_process(1000, 0.5, 1)
println(autocorr3(Xt, [0, 5]))

[0.9999999999999989, 0.355809435236412, -0.04230484739543982, -0.011683786561724356, -0.004341784406409456, -0.04645480657901478]


In [23]:
function autocorr5(X, lag)
    n = length(X)
    return [2/((n - h)*(n - h - 1)) * sum([sign((X[i] - X[j]) * (X[i + h] - X[j + h]))
                for i in 1:n - h for j in 1:i]) for h in lag[1]:lag[2]]
end

autocorr5 (generic function with 1 method)

In [24]:
Xt = MA1_process(1000, 0.5, 1)
println(autocorr5(Xt, [0, 5]))

[0.9999999999999999, 0.23674375778584195, -0.023559656926691902, -0.028084252758274822, -0.030437327198240195, 0.0015712364640101918]


####################################

In [25]:
function autocorr6(X, lag)
    n = length(X)
    mediana = median(X)
    acf = [1/(n - h) * sum([sign(i) for i in ((X[1:end-h] .- mediana) .*
                (X[1+h:end] .- mediana))]) for h in lag[1] + 1:lag[2]]
    return Float64[i==1 ? 1 : acf[i-1] for i in 1:length(acf) + 1]
end

autocorr6 (generic function with 1 method)

In [26]:
Xt = MA1_process(1000, 0.5, 1)
println(autocorr6(Xt, [0, 5]))

[1.0, 0.25925925925925924, -0.022044088176352703, 0.02106318956870612, 0.08032128514056225, 0.009045226130653268]


##############################

In [27]:
function zmatrix(X, k)
    n = length(X)
    Zk = zeros(k + 1, n + k)
    for j in 1:k+1
        Zk[j, j:j+n-1] = X[1:end]
    end
    return Zk
end

zmatrix (generic function with 1 method)

In [28]:
function gamma_k(X, k)
    z_matr = zmatrix(X, k)
    return (z_matr * transpose(z_matr))/length(X)
end

gamma_k (generic function with 1 method)

In [29]:
function Xi(X, k, i, j)
    gamm = gamma_k(X, k)
    return gamm[i, j]/sqrt(gamm[i, i] * gamm[j, j])
end
Xi(Xt, 10, 3, 4)

0.415476220559682

In [30]:
function autocorr7(X, lag)
    k = lag[2]
    return [1/(k - h + 1)*sum([Xi(X, k, i, i + h) for i in 1:k - h + 1]) for h in lag[1]:lag[2]]
end
Xt = MA1_process(1000, 0.5, 1)
println(autocorr7(Xt, [0, 5]))

[1.0, 0.40795681998145295, 0.03025049584994323, 0.028281308689702363, 0.01018063033238143, 0.002831246144638307]


In [31]:
function autocorr8(X, lag)
    return [(var(X[1+h:end].+X[1:end-h]) - var(X[1+h:end] .- X[1:end-h]))/(var(X[1+h:end].+X[1:end-h]) + var(X[1+h:end] .- X[1:end-h]))
            for h in lag[1]:lag[2]]
end
Xt = MA1_process(1000, 0.5, 1)
println(autocorr8(Xt, [0, 5]))

[1.0, 0.4192308238693258, -0.047141023264409575, -0.09644359567437308, -0.061441243626334296, -0.015689828827819455]


In [32]:
function MAD2(X, cons=1.4826)
    return (cons * median( abs.(X .- median(X)) ))^2
end

function IQR2(X, cons=0.7413)
    return (cons * (quantile(X, 0.75) - quantile(X, 0.25)))^2
end

IQR2 (generic function with 2 methods)

In [33]:
function autocorr9(X, lag)
    return [(MAD2(X[1+h:end].+X[1:end-h]) - MAD2(X[1+h:end] .- X[1:end-h]))/(MAD2(X[1+h:end].+X[1:end-h]) + MAD2(X[1+h:end] .- X[1:end-h]))
            for h in lag[1]:lag[2]]
end

autocorr9 (generic function with 1 method)

In [34]:
function autocorr10(X, lag)
    return [(IQR2(X[1+h:end].+X[1:end-h]) - IQR2(X[1+h:end] .- X[1:end-h]))/(IQR2(X[1+h:end].+X[1:end-h]) + IQR2(X[1+h:end] .- X[1:end-h]))
            for h in lag[1]:lag[2]]
end

autocorr10 (generic function with 1 method)

##################################

In [35]:
n = [100, 500, 1000]
p = [0.01, 0.02, 0.03, 0.04, 0.05, 0.06, 0.07]
a = [3, 10, 20, 30, 40, 50]

6-element Vector{Int64}:
  3
 10
 20
 30
 40
 50

In [36]:
global teta = 0.5
function comparison_emp_acf_theo_acf(acf, name)
    for j in n
        for k in 1:length(a)
             for l in 1:length(p)
                 Yt = [MA1_process(j, teta, 1) + Zt(p[l], a[k], j) for i in 1:100]
                 name == "autocorr1" ? emp_acf = [acf(Yt[i], [0, 5], 2.2191) for i in 1:length(Yt)] : (name == "autocorr2" ? emp_acf = [acf(Yt[i], [0, 5], 0.015) for i in 1:length(Yt)] : emp_acf = [acf(Yt[i], [0, 5]) for i in 1:length(Yt)])
                 emp_acf = mean(emp_acf)
                 global text = ""
                 for i in emp_acf
                     global text = text * string(i) * "\n"
                 end
                 open(string(@__DIR__) * "/dane/dla_zwyklego_modelu_MA1/" * name * "_n" * string(j) * "_a" * string(a[k]) * "_p" * string(p[l]) * ".txt", "w") do f
                     write(f, text)
                 end
             end
         end
     end
 end

comparison_emp_acf_theo_acf (generic function with 1 method)

In [37]:
@time begin
    comparison_emp_acf_theo_acf(autocorr, "autocorr")
end

  3.425544 seconds (2.05 M allocations: 3.064 GiB, 16.87% gc time, 23.41% compilation time)


In [38]:
@time begin
    comparison_emp_acf_theo_acf(autocorr1, "autocorr1")
end

4344.559937 seconds (4.80 M allocations: 1.151 TiB, 2.02% gc time, 0.00% compilation time)


In [39]:
@time begin
    comparison_emp_acf_theo_acf(autocorr2, "autocorr2")
end

3325.860912 seconds (284.05 M allocations: 1.659 TiB, 1.37% gc time, 0.00% compilation time)


In [40]:
@time begin
    comparison_emp_acf_theo_acf(autocorr3, "autocorr3")
end

 60.479694 seconds (376.35 k allocations: 907.812 MiB, 0.08% gc time, 0.21% compilation time)


In [41]:
@time begin
    comparison_emp_acf_theo_acf(autocorr4, "autocorr4")
end

  1.324107 seconds (502.36 k allocations: 1.399 GiB, 5.48% gc time, 3.59% compilation time)


In [42]:
@time begin
    comparison_emp_acf_theo_acf(autocorr5, "autocorr5")
end

264.894304 seconds (1.08 M allocations: 176.768 GiB, 5.34% gc time, 0.01% compilation time)


In [43]:
@time begin
    comparison_emp_acf_theo_acf(autocorr6, "autocorr6")
end

  0.990850 seconds (477.15 k allocations: 1.297 GiB, 9.89% gc time, 4.50% compilation time)


In [44]:
@time begin
    comparison_emp_acf_theo_acf(autocorr7, "autocorr7")
end

  6.870031 seconds (2.83 M allocations: 13.291 GiB, 7.92% gc time, 0.71% compilation time)


In [45]:
@time begin
    comparison_emp_acf_theo_acf(autocorr8, "autocorr8")
end

  1.943561 seconds (1.11 M allocations: 3.919 GiB, 16.15% gc time, 3.18% compilation time)


In [46]:
@time begin
    comparison_emp_acf_theo_acf(autocorr9, "autocorr9")
end

  8.031506 seconds (2.30 M allocations: 7.636 GiB, 5.44% gc time, 5.17% compilation time)


In [47]:
@time begin
    comparison_emp_acf_theo_acf(autocorr10, "autocorr10")
end

  6.995282 seconds (2.01 M allocations: 6.402 GiB, 5.09% gc time, 4.60% compilation time)


### Dla modelu ARMA

In [48]:
arma_pack = pyimport("statsmodels.tsa.arima_process")
np = pyimport("numpy")
ar4 = np.array([1, 0.33, 0.5, 0.86, 0.34])
ma4 = np.array([1, 0.9, 0.3, 0.12, 0.94])
global teta = 0.5
function comparison_emp_acf_theo_acf_ARMA44(acf, name)
    for j in n
        for k in 1:length(a)
             for l in 1:length(p)
                 Yt = [arma_pack.ArmaProcess(ar4, ma4).generate_sample(nsample=j) + Zt(p[l], a[k], j) for i in 1:100]
                 name == "autocorr1" ? emp_acf = [acf(Yt[i], [0, 5], 2.2191) for i in 1:length(Yt)] : (name == "autocorr2" ? emp_acf = [acf(Yt[i], [0, 5], 0.015) for i in 1:length(Yt)] : emp_acf = [acf(Yt[i], [0, 5]) for i in 1:length(Yt)])
                 emp_acf = mean(emp_acf)
                 global text = ""
                 for i in emp_acf
                     global text = text * string(i) * "\n"
                 end
                 open(string(@__DIR__) * "/dane/dla_modelu_arma44/" * name * "_n" * string(j) * "_a" * string(a[k]) * "_p" * string(p[l]) * ".txt", "w") do f
                     write(f, text)
                 end
             end
         end
     end
 end

comparison_emp_acf_theo_acf_ARMA44 (generic function with 1 method)

In [49]:
@time begin
    comparison_emp_acf_theo_acf_ARMA44(autocorr, "autocorr")
end

  3.617565 seconds (2.28 M allocations: 3.022 GiB, 7.35% gc time, 15.55% compilation time)


In [50]:
@time begin
    comparison_emp_acf_theo_acf_ARMA44(autocorr1, "autocorr1")
end

4374.247891 seconds (5.49 M allocations: 1.151 TiB, 1.97% gc time, 0.00% compilation time)


In [51]:
@time begin
    comparison_emp_acf_theo_acf_ARMA44(autocorr2, "autocorr2")
end

2034.337192 seconds (284.74 M allocations: 1.659 TiB, 2.35% gc time, 0.00% compilation time)


In [52]:
@time begin
    comparison_emp_acf_theo_acf_ARMA44(autocorr3, "autocorr3")
end

 43.278418 seconds (1.07 M allocations: 889.300 MiB, 0.12% gc time, 0.09% compilation time)


In [53]:
@time begin
    comparison_emp_acf_theo_acf_ARMA44(autocorr4, "autocorr4")
end

  2.207511 seconds (1.19 M allocations: 1.381 GiB, 3.64% gc time, 1.81% compilation time)


In [54]:
@time begin
    comparison_emp_acf_theo_acf_ARMA44(autocorr5, "autocorr5")
end

186.565389 seconds (1.77 M allocations: 176.750 GiB, 6.28% gc time, 0.02% compilation time)


In [55]:
@time begin
    comparison_emp_acf_theo_acf_ARMA44(autocorr6, "autocorr6")
end

  1.542281 seconds (1.17 M allocations: 1.279 GiB, 5.03% gc time, 2.36% compilation time)


In [56]:
@time begin
    comparison_emp_acf_theo_acf_ARMA44(autocorr7, "autocorr7")
end

  5.829905 seconds (3.52 M allocations: 13.273 GiB, 7.45% gc time, 0.59% compilation time)


In [57]:
@time begin
    comparison_emp_acf_theo_acf_ARMA44(autocorr8, "autocorr8")
end

  2.126195 seconds (1.80 M allocations: 3.901 GiB, 8.78% gc time, 1.50% compilation time)


In [58]:
@time begin
    comparison_emp_acf_theo_acf_ARMA44(autocorr9, "autocorr9")
end

  4.969982 seconds (2.70 M allocations: 7.604 GiB, 6.73% gc time, 0.69% compilation time)


In [59]:
@time begin
    comparison_emp_acf_theo_acf_ARMA44(autocorr10, "autocorr10")
end

  4.673709 seconds (2.40 M allocations: 6.370 GiB, 6.00% gc time, 0.66% compilation time)


### Dla rozkładu t-studenta z 4 stopniami swobody

In [60]:
global teta = 0.5

function MA1_process_t_distribution(n, theta, freedom_degrees)
    Z = rand(TDist(freedom_degrees), n+1)
    return [Z[i+1] + theta*Z[i] for i in 1:n]
end

function comparison_emp_acf_theo_acf_t(acf, name, df)
    for j in n
        for k in 1:length(a)
             for l in 1:length(p)
                 Yt = [MA1_process_t_distribution(j, teta, df) + Zt(p[l], a[k], j) for i in 1:100]
                 name == "autocorr1" ? emp_acf = [acf(Yt[i], [0, 5], 2.2191) for i in 1:length(Yt)] : (name == "autocorr2" ? emp_acf = [acf(Yt[i], [0, 5], 0.015) for i in 1:length(Yt)] : emp_acf = [acf(Yt[i], [0, 5]) for i in 1:length(Yt)])
                 emp_acf = mean(emp_acf)
                 global text = ""
                 for i in emp_acf
                     global text = text * string(i) * "\n"
                 end
                 open(string(@__DIR__) * "/dane/dla_modelu_t4/" * name * "_n" * string(j) * "_a" * string(a[k]) * "_p" * string(p[l]) * ".txt", "w") do f
                     write(f, text)
                 end
             end
         end
     end
 end

comparison_emp_acf_theo_acf_t (generic function with 1 method)

In [61]:
@time begin
    comparison_emp_acf_theo_acf_t(autocorr, "autocorr", 4)
end

  1.578441 seconds (1.65 M allocations: 3.042 GiB, 9.78% gc time, 16.69% compilation time)


In [62]:
@time begin
    comparison_emp_acf_theo_acf_t(autocorr1, "autocorr1", 4)
end

3757.062761 seconds (4.80 M allocations: 1.151 TiB, 1.98% gc time, 0.00% compilation time)


In [63]:
@time begin
    comparison_emp_acf_theo_acf_t(autocorr2, "autocorr2", 4)
end

2329.740935 seconds (284.05 M allocations: 1.659 TiB, 2.00% gc time, 0.00% compilation time)


In [64]:
@time begin
    comparison_emp_acf_theo_acf_t(autocorr3, "autocorr3", 4)
end

 40.455277 seconds (376.35 k allocations: 907.810 MiB, 0.07% gc time, 0.08% compilation time)


In [65]:
@time begin
    comparison_emp_acf_theo_acf_t(autocorr4, "autocorr4", 4)
end

  1.165147 seconds (502.35 k allocations: 1.399 GiB, 3.84% gc time, 2.75% compilation time)


In [66]:
@time begin
    comparison_emp_acf_theo_acf_t(autocorr5, "autocorr5", 4)
end

181.447025 seconds (1.08 M allocations: 176.768 GiB, 5.28% gc time, 0.02% compilation time)


In [67]:
@time begin
    comparison_emp_acf_theo_acf_t(autocorr6, "autocorr6", 4)
end

  0.941111 seconds (477.15 k allocations: 1.297 GiB, 7.72% gc time, 3.33% compilation time)


In [68]:
@time begin
    comparison_emp_acf_theo_acf_t(autocorr7, "autocorr7", 4)
end

  5.155469 seconds (2.83 M allocations: 13.291 GiB, 8.18% gc time, 0.56% compilation time)


In [69]:
@time begin
    comparison_emp_acf_theo_acf_t(autocorr8, "autocorr8", 4)
end

  1.610114 seconds (1.11 M allocations: 3.919 GiB, 12.75% gc time, 1.74% compilation time)


In [70]:
@time begin
    comparison_emp_acf_theo_acf_t(autocorr9, "autocorr9", 4)
end

  5.471066 seconds (2.01 M allocations: 7.622 GiB, 6.45% gc time, 0.51% compilation time)


In [71]:
@time begin
    comparison_emp_acf_theo_acf_t(autocorr10, "autocorr10", 4)
end

  4.695529 seconds (1.71 M allocations: 6.388 GiB, 5.78% gc time, 0.59% compilation time)


### Dla rozkładu t-studenta z 20 stopniami swobody

In [72]:
function comparison_emp_acf_theo_acf_t20(acf, name, df)
    for j in n
        for k in 1:length(a)
             for l in 1:length(p)
                 Yt = [MA1_process_t_distribution(j, teta, df) + Zt(p[l], a[k], j) for i in 1:100]
                 name == "autocorr1" ? emp_acf = [acf(Yt[i], [0, 5], 2.2191) for i in 1:length(Yt)] : (name == "autocorr2" ? emp_acf = [acf(Yt[i], [0, 5], 0.015) for i in 1:length(Yt)] : emp_acf = [acf(Yt[i], [0, 5]) for i in 1:length(Yt)])
                 emp_acf = mean(emp_acf)
                 global text = ""
                 for i in emp_acf
                     global text = text * string(i) * "\n"
                 end
                 open(string(@__DIR__) * "/dane/dla_modelu_t20/" * name * "_n" * string(j) * "_a" * string(a[k]) * "_p" * string(p[l]) * ".txt", "w") do f
                     write(f, text)
                 end
             end
         end
     end
 end

comparison_emp_acf_theo_acf_t20 (generic function with 1 method)

In [73]:
@time begin
    comparison_emp_acf_theo_acf_t20(autocorr, "autocorr", 20)
end

  1.164260 seconds (1.10 M allocations: 3.014 GiB, 12.45% gc time, 6.54% compilation time)


In [74]:
@time begin
    comparison_emp_acf_theo_acf_t20(autocorr1, "autocorr1", 20)
end

3881.037213 seconds (4.80 M allocations: 1.151 TiB, 2.08% gc time, 0.00% compilation time)


In [75]:
@time begin
    comparison_emp_acf_theo_acf_t20(autocorr2, "autocorr2", 20)
end

2435.809778 seconds (284.05 M allocations: 1.659 TiB, 2.06% gc time, 0.00% compilation time)


In [76]:
@time begin
    comparison_emp_acf_theo_acf_t20(autocorr3, "autocorr3", 20)
end

 42.163106 seconds (376.35 k allocations: 907.812 MiB, 0.07% gc time, 0.08% compilation time)


In [77]:
@time begin
    comparison_emp_acf_theo_acf_t20(autocorr4, "autocorr4", 20)
end

  1.129966 seconds (502.35 k allocations: 1.399 GiB, 4.34% gc time, 3.64% compilation time)


In [78]:
@time begin
    comparison_emp_acf_theo_acf_t20(autocorr5, "autocorr5", 20)
end

190.028429 seconds (1.08 M allocations: 176.768 GiB, 5.89% gc time, 0.02% compilation time)


In [79]:
@time begin
    comparison_emp_acf_theo_acf_t20(autocorr6, "autocorr6", 20)
end

  0.892679 seconds (477.15 k allocations: 1.297 GiB, 8.48% gc time, 4.11% compilation time)


In [80]:
@time begin
    comparison_emp_acf_theo_acf_t20(autocorr7, "autocorr7", 20)
end

  5.490268 seconds (2.83 M allocations: 13.291 GiB, 9.00% gc time, 0.82% compilation time)


In [81]:
@time begin
    comparison_emp_acf_theo_acf_t20(autocorr8, "autocorr8", 20)
end

  1.646365 seconds (1.11 M allocations: 3.919 GiB, 13.18% gc time, 1.84% compilation time)


In [82]:
@time begin
    comparison_emp_acf_theo_acf_t20(autocorr9, "autocorr9", 20)
end

  5.584317 seconds (2.01 M allocations: 7.622 GiB, 6.78% gc time, 0.54% compilation time)


In [83]:
@time begin
    comparison_emp_acf_theo_acf_t20(autocorr10, "autocorr10", 20)
end

  4.740370 seconds (1.71 M allocations: 6.388 GiB, 6.33% gc time, 0.63% compilation time)


## Dla zwykłego MA1 z ciągłym szumem

In [84]:
n = [100, 500, 1000]
sigm = [0.2, 0.75, 1.5]

3-element Vector{Float64}:
 0.2
 0.75
 1.5

In [85]:
global teta = 0.5
function comparison_emp_acf_theo_acf_continous_noise(acf, name)
    for j in n
        for k in sigm
            Yt = [MA1_process(j, teta, 1) + rand(Normal(0, k), j) for i in 1:100]
            name == "autocorr1" ? emp_acf = [acf(Yt[i], [0, 5], 2.2191) for i in 1:length(Yt)] : (name == "autocorr2" ? emp_acf = [acf(Yt[i], [0, 5], 0.015) for i in 1:length(Yt)] : emp_acf = [acf(Yt[i], [0, 5]) for i in 1:length(Yt)])
            emp_acf = mean(emp_acf)
            global text = ""
            for i in emp_acf
                global text = text * string(i) * "\n"
            end
            open(string(@__DIR__) * "/dane/dla_zwyklego_modelu_MA1_z_ciaglym_szumem/" * name * "_n" * string(j) * "_sigma" * string(k) * ".txt", "w") do f
                write(f, text)
            end
         end
     end
 end

comparison_emp_acf_theo_acf_continous_noise (generic function with 1 method)

In [86]:
@time begin
    comparison_emp_acf_theo_acf_continous_noise(autocorr, "autocorr")
end

  0.158599 seconds (285.58 k allocations: 231.935 MiB, 9.43% gc time, 62.03% compilation time)


In [87]:
@time begin
    comparison_emp_acf_theo_acf_continous_noise(autocorr1, "autocorr1")
end

283.770791 seconds (418.50 k allocations: 84.222 GiB, 1.86% gc time, 0.01% compilation time)


In [88]:
@time begin
    comparison_emp_acf_theo_acf_continous_noise(autocorr2, "autocorr2")
end

177.173518 seconds (20.37 M allocations: 121.315 GiB, 2.62% gc time, 0.02% compilation time)


In [89]:
@time begin
    comparison_emp_acf_theo_acf_continous_noise(autocorr3, "autocorr3")
end

  2.988686 seconds (102.52 k allocations: 69.046 MiB, 0.28% gc time, 1.14% compilation time)


In [90]:
@time begin
    comparison_emp_acf_theo_acf_continous_noise(autocorr4, "autocorr4")
end

  0.099692 seconds (111.52 k allocations: 106.527 MiB, 8.83% gc time, 34.03% compilation time)


In [91]:
@time begin
    comparison_emp_acf_theo_acf_continous_noise(autocorr5, "autocorr5")
end

 13.737132 seconds (152.08 k allocations: 12.630 GiB, 7.92% gc time, 0.22% compilation time)


In [92]:
@time begin
    comparison_emp_acf_theo_acf_continous_noise(autocorr6, "autocorr6")
end

  0.097232 seconds (109.72 k allocations: 99.038 MiB, 8.70% gc time, 35.23% compilation time)


In [93]:
@time begin
    comparison_emp_acf_theo_acf_continous_noise(autocorr7, "autocorr7")
end

  0.411003 seconds (277.20 k allocations: 976.360 MiB, 11.36% gc time, 9.03% compilation time)


In [94]:
@time begin
    comparison_emp_acf_theo_acf_continous_noise(autocorr8, "autocorr8")
end

  0.140496 seconds (153.81 k allocations: 290.852 MiB, 16.97% gc time, 31.59% compilation time)


In [95]:
@time begin
    comparison_emp_acf_theo_acf_continous_noise(autocorr9, "autocorr9")
end

  0.421598 seconds (218.62 k allocations: 561.721 MiB, 6.96% gc time, 9.21% compilation time)


In [96]:
@time begin
    comparison_emp_acf_theo_acf_continous_noise(autocorr10, "autocorr10")
end

  0.333696 seconds (197.01 k allocations: 471.430 MiB, 6.75% gc time, 9.02% compilation time)


### Dla ARMA z dodanym szumem ciągłym

In [97]:
ar4 = np.array([1, 0.33, 0.5, 0.86, 0.34])
ma4 = np.array([1, 0.9, 0.3, 0.12, 0.94])
global teta = 0.5
function comparison_emp_acf_theo_acf_ARMA44_continous_noise(acf, name)
    for j in n
        for k in sigm
            Yt = [arma_pack.ArmaProcess(ar4, ma4).generate_sample(nsample=j) + rand(Normal(0, k), j) for i in 1:100]
            name == "autocorr1" ? emp_acf = [acf(Yt[i], [0, 5], 2.2191) for i in 1:length(Yt)] : (name == "autocorr2" ? emp_acf = [acf(Yt[i], [0, 5], 0.015) for i in 1:length(Yt)] : emp_acf = [acf(Yt[i], [0, 5]) for i in 1:length(Yt)])
            emp_acf = mean(emp_acf)
            global text = ""
            for i in emp_acf
                global text = text * string(i) * "\n"
            end
            open(string(@__DIR__) * "/dane/dla_modelu_arma44_z_ciaglym_szumem/" * name * "_n" * string(j) * "_sigma" * string(k) * ".txt", "w") do f
                write(f, text)
            end
         end
     end
 end

comparison_emp_acf_theo_acf_ARMA44_continous_noise (generic function with 1 method)

In [98]:
@time begin
    comparison_emp_acf_theo_acf_ARMA44_continous_noise(autocorr, "autocorr")
end

  0.226744 seconds (293.71 k allocations: 228.321 MiB, 8.31% gc time, 41.71% compilation time)


In [99]:
@time begin
    comparison_emp_acf_theo_acf_ARMA44_continous_noise(autocorr1, "autocorr1")
end

249.964296 seconds (467.71 k allocations: 84.220 GiB, 2.26% gc time, 0.01% compilation time)


In [100]:
@time begin
    comparison_emp_acf_theo_acf_ARMA44_continous_noise(autocorr2, "autocorr2")
end

140.437438 seconds (20.42 M allocations: 121.313 GiB, 3.31% gc time, 0.02% compilation time)


In [101]:
@time begin
    comparison_emp_acf_theo_acf_ARMA44_continous_noise(autocorr3, "autocorr3")
end

  3.069663 seconds (151.73 k allocations: 67.747 MiB, 0.23% gc time, 1.36% compilation time)


In [102]:
@time begin
    comparison_emp_acf_theo_acf_ARMA44_continous_noise(autocorr4, "autocorr4")
end

  0.155700 seconds (160.72 k allocations: 105.162 MiB, 5.01% gc time, 21.53% compilation time)


In [103]:
@time begin
    comparison_emp_acf_theo_acf_ARMA44_continous_noise(autocorr5, "autocorr5")
end

 13.611272 seconds (201.30 k allocations: 12.629 GiB, 5.99% gc time, 0.23% compilation time)


In [104]:
@time begin
    comparison_emp_acf_theo_acf_ARMA44_continous_noise(autocorr6, "autocorr6")
end

  0.143701 seconds (158.94 k allocations: 97.763 MiB, 5.70% gc time, 23.59% compilation time)


In [105]:
@time begin
    comparison_emp_acf_theo_acf_ARMA44_continous_noise(autocorr7, "autocorr7")
end

  0.445863 seconds (326.41 k allocations: 975.019 MiB, 10.36% gc time, 6.68% compilation time)


In [106]:
@time begin
    comparison_emp_acf_theo_acf_ARMA44_continous_noise(autocorr8, "autocorr8")
end

  0.219022 seconds (203.02 k allocations: 289.554 MiB, 10.52% gc time, 16.49% compilation time)


In [107]:
@time begin
    comparison_emp_acf_theo_acf_ARMA44_continous_noise(autocorr9, "autocorr9")
end

  0.376218 seconds (267.83 k allocations: 560.380 MiB, 7.89% gc time, 8.47% compilation time)


In [108]:
@time begin
    comparison_emp_acf_theo_acf_ARMA44_continous_noise(autocorr10, "autocorr10")
end

  0.405808 seconds (246.22 k allocations: 470.133 MiB, 9.52% gc time, 7.68% compilation time)


### Dla rozkładu t-studenta z 4 stopniami swobody z ciągłym szumem

In [109]:
function comparison_emp_acf_theo_acf_t_continous_noise(acf, name, df)
    for j in n
        for k in sigm
            Yt = [MA1_process_t_distribution(j, teta, df) + rand(Normal(0, k), j) for i in 1:100]
            name == "autocorr1" ? emp_acf = [acf(Yt[i], [0, 5], 2.2191) for i in 1:length(Yt)] : (name == "autocorr2" ? emp_acf = [acf(Yt[i], [0, 5], 0.015) for i in 1:length(Yt)] : emp_acf = [acf(Yt[i], [0, 5]) for i in 1:length(Yt)])
            emp_acf = mean(emp_acf)
            global text = ""
            for i in emp_acf
                global text = text * string(i) * "\n"
            end
            open(string(@__DIR__) * "/dane/dla_modelu_t4_z_ciaglym_szumem/" * name * "_n" * string(j) * "_sigma" * string(k) * ".txt", "w") do f
                write(f, text)
            end
         end
     end
 end

comparison_emp_acf_theo_acf_t_continous_noise (generic function with 1 method)

In [110]:
@time begin
    comparison_emp_acf_theo_acf_t_continous_noise(autocorr, "autocorr", 4)
end

  0.185495 seconds (243.08 k allocations: 229.544 MiB, 11.14% gc time, 48.92% compilation time)


In [111]:
@time begin
    comparison_emp_acf_theo_acf_t_continous_noise(autocorr1, "autocorr1", 4)
end

280.313600 seconds (418.50 k allocations: 84.222 GiB, 2.02% gc time, 0.01% compilation time)


In [112]:
@time begin
    comparison_emp_acf_theo_acf_t_continous_noise(autocorr2, "autocorr2", 4)
end

175.625478 seconds (20.37 M allocations: 121.315 GiB, 2.64% gc time, 0.02% compilation time)


In [113]:
@time begin
    comparison_emp_acf_theo_acf_t_continous_noise(autocorr3, "autocorr3", 4)
end

  3.015468 seconds (102.52 k allocations: 69.045 MiB, 0.22% gc time, 1.11% compilation time)


In [114]:
@time begin
    comparison_emp_acf_theo_acf_t_continous_noise(autocorr4, "autocorr4", 4)
end

  0.129031 seconds (111.52 k allocations: 106.526 MiB, 5.38% gc time, 27.41% compilation time)


In [115]:
@time begin
    comparison_emp_acf_theo_acf_t_continous_noise(autocorr5, "autocorr5", 4)
end

 13.602147 seconds (152.08 k allocations: 12.630 GiB, 5.99% gc time, 0.23% compilation time)


In [116]:
@time begin
    comparison_emp_acf_theo_acf_t_continous_noise(autocorr6, "autocorr6", 4)
end

  0.100173 seconds (109.72 k allocations: 99.037 MiB, 10.46% gc time, 33.77% compilation time)


In [117]:
@time begin
    comparison_emp_acf_theo_acf_t_continous_noise(autocorr7, "autocorr7", 4)
end

In [118]:
@time begin
    comparison_emp_acf_theo_acf_t_continous_noise(autocorr8, "autocorr8", 4)
end

  0.405335 seconds (277.20 k allocations: 976.360 MiB, 10.80% gc time, 7.84% compilation time)
  0.199981 seconds (153.81 k allocations: 290.851 MiB, 17.73% gc time, 15.07% compilation time)


In [119]:
@time begin
    comparison_emp_acf_theo_acf_t_continous_noise(autocorr9, "autocorr9", 4)
end

  0.434219 seconds (218.62 k allocations: 561.721 MiB, 6.99% gc time, 8.72% compilation time)


In [120]:
@time begin
    comparison_emp_acf_theo_acf_t_continous_noise(autocorr10, "autocorr10", 4)
end

  0.365159 seconds (197.01 k allocations: 471.430 MiB, 6.71% gc time, 8.21% compilation time)


### Dla rozkładu t-studenta z 20 stopniami swobody z ciaglym szumem

In [121]:
function comparison_emp_acf_theo_acf_t20_continous_noise(acf, name, df)
    for j in n
        for k in sigm
            Yt = [MA1_process_t_distribution(j, teta, df) + rand(Normal(0, k), j) for i in 1:100]
            name == "autocorr1" ? emp_acf = [acf(Yt[i], [0, 5], 2.2191) for i in 1:length(Yt)] : (name == "autocorr2" ? emp_acf = [acf(Yt[i], [0, 5], 0.015) for i in 1:length(Yt)] : emp_acf = [acf(Yt[i], [0, 5]) for i in 1:length(Yt)])
            emp_acf = mean(emp_acf)
            global text = ""
            for i in emp_acf
                global text = text * string(i) * "\n"
            end
            open(string(@__DIR__) * "/dane/dla_modelu_t20_z_ciaglym_szumem/" * name * "_n" * string(j) * "_sigma" * string(k) * ".txt", "w") do f
                write(f, text)
            end
         end
     end
 end

comparison_emp_acf_theo_acf_t20_continous_noise (generic function with 1 method)

In [122]:
@time begin
    comparison_emp_acf_theo_acf_t20_continous_noise(autocorr, "autocorr", 20)
end

  0.180639 seconds (243.08 k allocations: 229.564 MiB, 11.37% gc time, 50.83% compilation time)


In [123]:
@time begin
    comparison_emp_acf_theo_acf_t20_continous_noise(autocorr1, "autocorr1", 20)
end

283.184290 seconds (418.50 k allocations: 84.222 GiB, 1.99% gc time, 0.01% compilation time)


In [124]:
@time begin
    comparison_emp_acf_theo_acf_t20_continous_noise(autocorr2, "autocorr2", 20)
end

179.225871 seconds (20.37 M allocations: 121.315 GiB, 2.56% gc time, 0.02% compilation time)


In [125]:
@time begin
    comparison_emp_acf_theo_acf_t20_continous_noise(autocorr3, "autocorr3", 20)
end

  3.052983 seconds (102.52 k allocations: 69.045 MiB, 0.22% gc time, 1.31% compilation time)


In [126]:
@time begin
    comparison_emp_acf_theo_acf_t20_continous_noise(autocorr4, "autocorr4", 20)
end

  0.118565 seconds (111.52 k allocations: 106.530 MiB, 5.99% gc time, 29.49% compilation time)


In [127]:
@time begin
    comparison_emp_acf_theo_acf_t20_continous_noise(autocorr5, "autocorr5", 20)
end

 13.670483 seconds (152.08 k allocations: 12.630 GiB, 5.81% gc time, 0.24% compilation time)


In [128]:
@time begin
    comparison_emp_acf_theo_acf_t20_continous_noise(autocorr6, "autocorr6", 20)
end

  0.107328 seconds (109.72 k allocations: 99.048 MiB, 10.22% gc time, 31.30% compilation time)


In [129]:
@time begin
    comparison_emp_acf_theo_acf_t20_continous_noise(autocorr7, "autocorr7", 20)
end

  0.412085 seconds (277.20 k allocations: 976.360 MiB, 11.08% gc time, 7.47% compilation time)


In [130]:
@time begin
    comparison_emp_acf_theo_acf_t20_continous_noise(autocorr8, "autocorr8", 20)
end

  0.141956 seconds (153.81 k allocations: 290.851 MiB, 10.68% gc time, 21.95% compilation time)


In [131]:
@time begin
    comparison_emp_acf_theo_acf_t20_continous_noise(autocorr9, "autocorr9", 20)
end

  0.447078 seconds (218.62 k allocations: 561.724 MiB, 8.12% gc time, 8.81% compilation time)


In [132]:
@time begin
    comparison_emp_acf_theo_acf_t20_continous_noise(autocorr10, "autocorr10", 20)
end

  0.358292 seconds (197.01 k allocations: 471.430 MiB, 6.48% gc time, 8.51% compilation time)
